## Initialization

In [1]:
import tensorflow.keras as K
from training import train_model, DIM

callback = [K.callbacks.EarlyStopping(
    monitor = 'val_loss',
    min_delta = 0.1,
    verbose = 1
)]

---

## Model 1

In [4]:
class Conv1D(K.layers.Layer):
    def __init__(self, _output, kernel_size = 3, strides = 1, dilation_rate = 1, padding = 'VALID'):
        super(Conv1D, self).__init__()
        
        self.conv = K.layers.Conv1D(
            _output,
            kernel_size = kernel_size,
            strides = strides,
            dilation_rate = dilation_rate,
            padding = padding,
            activation = 'relu'
        )
        
    def call(self, parameters):
        return self.conv(parameters)

In [5]:
model = K.models.Sequential([
    K.layers.Input(shape = DIM),
    Conv1D(128, strides = 2),
    Conv1D(256, strides = 2),
    Conv1D(512, strides = 2),
    Conv1D(1024, strides = 2),
    K.layers.GlobalMaxPooling1D(),
    K.layers.Dense(12, activation = 'softmax')
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_8 (Conv1D)            (None, 214, 128)          32384     
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 106, 256)          98560     
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 52, 512)           393728    
_________________________________________________________________
conv1d_14 (Conv1D)           (None, 25, 1024)          1573888   
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 12)                12300     
Total params: 2,110,860
Trainable params: 2,110,860
Non-trainable params: 0
____________________________________________

In [6]:
train_model(model, callbacks = callback)

Epoch 1/10
1115/1115 [==============================] - 400s 356ms/step - loss: 1.7882 - accuracy: 0.3922 - val_loss: 2.2494 - val_accuracy: 0.2964
Epoch 2/10
1115/1115 [==============================] - 414s 372ms/step - loss: 1.5218 - accuracy: 0.4958 - val_loss: 2.0372 - val_accuracy: 0.3560
Epoch 3/10
1115/1115 [==============================] - 366s 329ms/step - loss: 1.4471 - accuracy: 0.5234 - val_loss: 1.8092 - val_accuracy: 0.4310
Epoch 4/10
1115/1115 [==============================] - 427s 383ms/step - loss: 1.3554 - accuracy: 0.5537 - val_loss: 1.8071 - val_accuracy: 0.4076
Epoch 00004: early stopping


{'loss': [1.7019027471542358,
  1.525709867477417,
  1.4351662397384644,
  1.3704304695129395],
 'accuracy': [0.43110987544059753,
  0.49419844150543213,
  0.5296104550361633,
  0.5478278994560242],
 'val_loss': [2.249445676803589,
  2.037160634994507,
  1.809190034866333,
  1.8071439266204834],
 'val_accuracy': [0.29635417461395264,
  0.35598957538604736,
  0.4309895932674408,
  0.4075520932674408]}

---

## Model 2

In [2]:
class ResConv1D(K.layers.Layer):
    def __init__(self, _output, kernel_size = 3, strides = 1, dilation_rate = 1, padding = 'SAME'):
        super(ResConv1D, self).__init__()
        
        self.conv = K.layers.Conv1D(
            _output,
            kernel_size = kernel_size,
            strides = strides,
            dilation_rate = dilation_rate,
            padding = padding,
            activation = 'relu'
        )
        self.normalization = K.layers.BatchNormalization()
        
        self.T = K.layers.Conv1D(
            _output,
            kernel_size = kernel_size,
            padding = padding,
            activation = 'relu'
        )
        self.T_normalization = K.layers.BatchNormalization()
        
        self.upsample = K.layers.Conv1D(
            _output,
            kernel_size = 1,
            strides = strides,
            padding = padding,
            activation = 'relu'
        )
        
    def call(self, parameters):
        result = self.normalization(self.conv(parameters))
        result = self.T_normalization(self.T(result))
        result += self.upsample(parameters)
        return result

In [3]:
model = K.models.Sequential([
    K.layers.Input(shape = DIM),
    ResConv1D(128, strides = 2),
    ResConv1D(256, strides = 2),
    ResConv1D(512, strides = 2),
    ResConv1D(1024, strides = 2),
    K.layers.GlobalMaxPooling1D(),
    K.layers.Dense(256, activation = 'relu'),
    K.layers.Dense(12, activation = 'softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
res_conv1d (ResConv1D)       (None, 215, 128)          93568     
_________________________________________________________________
res_conv1d_1 (ResConv1D)     (None, 108, 256)          330496    
_________________________________________________________________
res_conv1d_2 (ResConv1D)     (None, 54, 512)           1316352   
_________________________________________________________________
res_conv1d_3 (ResConv1D)     (None, 27, 1024)          5254144   
_________________________________________________________________
global_max_pooling1d (Global (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               262400    
_________________________________________________________________
dense_1 (Dense)              (None, 12)                3

In [23]:
train_model(model, callbacks = callback)

Epoch 1/10
1115/1115 [==============================] - 393s 350ms/step - loss: 2.0909 - accuracy: 0.4015 - val_loss: 2.2105 - val_accuracy: 0.2842
Epoch 2/10
1115/1115 [==============================] - 400s 359ms/step - loss: 1.6267 - accuracy: 0.4670 - val_loss: 1.9675 - val_accuracy: 0.3866
Epoch 3/10
1115/1115 [==============================] - 413s 371ms/step - loss: 1.4477 - accuracy: 0.5283 - val_loss: 1.8239 - val_accuracy: 0.4206
Epoch 4/10
1115/1115 [==============================] - 405s 363ms/step - loss: 1.3905 - accuracy: 0.5460 - val_loss: 2.0393 - val_accuracy: 0.4267
Epoch 00004: early stopping


{'loss': [1.7598035335540771,
  1.5956388711929321,
  1.45634925365448,
  1.3819023370742798],
 'accuracy': [0.43249720335006714,
  0.4750981032848358,
  0.5248038172721863,
  0.5498458743095398],
 'val_loss': [2.21049165725708,
  1.9675161838531494,
  1.8238587379455566,
  2.03932523727417],
 'val_accuracy': [0.2842448055744171,
  0.38658854365348816,
  0.4205729067325592,
  0.4266926944255829]}

In [4]:
train_model(model, optimizer = K.optimizers.Adam(lr = 1e-2), callbacks = callback)

Epoch 1/10
1115/1115 [==============================] - 412s 365ms/step - loss: 5.1051 - accuracy: 0.2875 - val_loss: 2.1517 - val_accuracy: 0.3319
Epoch 2/10
1115/1115 [==============================] - 406s 365ms/step - loss: 1.8023 - accuracy: 0.3958 - val_loss: 2.4277 - val_accuracy: 0.3385
Epoch 00002: early stopping


{'loss': [2.5461575984954834, 1.7934486865997314],
 'accuracy': [0.33577635884284973, 0.39928531646728516],
 'val_loss': [2.1517043113708496, 2.4277191162109375],
 'val_accuracy': [0.33190104365348816, 0.3385416567325592]}

In [5]:
train_model(model, optimizer = K.optimizers.SGD(lr = 1e-3), callbacks = callback)

Epoch 1/10
1115/1115 [==============================] - 416s 371ms/step - loss: 1.9056 - accuracy: 0.3835 - val_loss: 2.0001 - val_accuracy: 0.3777
Epoch 2/10
1115/1115 [==============================] - 393s 353ms/step - loss: 1.8196 - accuracy: 0.3941 - val_loss: 1.9077 - val_accuracy: 0.3779
Epoch 00002: early stopping


{'loss': [1.8710236549377441, 1.821566104888916],
 'accuracy': [0.3877802789211273, 0.39680492877960205],
 'val_loss': [2.000094175338745, 1.9077296257019043],
 'val_accuracy': [0.37773436307907104, 0.3778645694255829]}

---

## Model 3

In [7]:
model = K.models.Sequential([
    K.layers.Input(shape = DIM),
    ResConv1D(128, strides = 2),
    ResConv1D(256, strides = 2),
    ResConv1D(512, strides = 2),
    ResConv1D(1024, strides = 2),
    ResConv1D(2048, strides = 2),
    ResConv1D(4096, strides = 2),
    K.layers.GlobalMaxPooling1D(),
    K.layers.Dense(256, activation = 'relu'),
    K.layers.Dense(12, activation = 'softmax')
])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
res_conv1d_9 (ResConv1D)     (None, 215, 128)          93568     
_________________________________________________________________
res_conv1d_10 (ResConv1D)    (None, 108, 256)          330496    
_________________________________________________________________
res_conv1d_11 (ResConv1D)    (None, 54, 512)           1316352   
_________________________________________________________________
res_conv1d_12 (ResConv1D)    (None, 27, 1024)          5254144   
_________________________________________________________________
res_conv1d_13 (ResConv1D)    (None, 14, 2048)          20994048  
_________________________________________________________________
res_conv1d_14 (ResConv1D)    (None, 7, 4096)           83931136  
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 4096)             

In [8]:
train_model(model, callbacks = callback)

Epoch 1/10
1115/1115 [==============================] - 439s 385ms/step - loss: 2.6689 - accuracy: 0.3610 - val_loss: 2.1161 - val_accuracy: 0.3012
Epoch 2/10
1115/1115 [==============================] - 418s 375ms/step - loss: 1.7394 - accuracy: 0.4283 - val_loss: 2.1292 - val_accuracy: 0.3229
Epoch 00002: early stopping


{'loss': [2.007978677749634, 1.7062395811080933],
 'accuracy': [0.3752942681312561, 0.44051289558410645],
 'val_loss': [2.116095542907715, 2.1292319297790527],
 'val_accuracy': [0.3011718690395355, 0.3229166567325592]}

---

## Model 4

In [9]:
model = K.models.Sequential([
    K.layers.Input(shape = DIM),
    ResConv1D(128, strides = 2),
    ResConv1D(256, strides = 2),
    ResConv1D(512, strides = 2),
    ResConv1D(1024, strides = 2),
    ResConv1D(2048, strides = 2),
    ResConv1D(4096, strides = 2),
    K.layers.Flatten(),
    K.layers.Dense(256, activation = 'relu'),
    K.layers.Dense(12, activation = 'softmax')
])

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
res_conv1d_15 (ResConv1D)    (None, 215, 128)          93568     
_________________________________________________________________
res_conv1d_16 (ResConv1D)    (None, 108, 256)          330496    
_________________________________________________________________
res_conv1d_17 (ResConv1D)    (None, 54, 512)           1316352   
_________________________________________________________________
res_conv1d_18 (ResConv1D)    (None, 27, 1024)          5254144   
_________________________________________________________________
res_conv1d_19 (ResConv1D)    (None, 14, 2048)          20994048  
_________________________________________________________________
res_conv1d_20 (ResConv1D)    (None, 7, 4096)           83931136  
_________________________________________________________________
flatten (Flatten)            (None, 28672)            

In [10]:
train_model(model, callbacks = callback)

Epoch 1/10
1115/1115 [==============================] - ETA: 0s - loss: 5.3057 - accuracy: 0.1624

KeyboardInterrupt: 

In [14]:
model = K.models.Sequential([
    K.layers.Input(shape = DIM),
    ResConv1D(128, dilation_rate = 2),
    ResConv1D(128, dilation_rate = 2),
    ResConv1D(256, dilation_rate = 2),
    ResConv1D(512, dilation_rate = 2),
    ResConv1D(1024, dilation_rate = 2),
    K.layers.GlobalMaxPooling1D(),
    K.layers.Dense(256, activation = 'relu'),
    K.layers.Dense(12, activation = 'softmax')
])

model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
res_conv1d_33 (ResConv1D)    (None, 430, 128)          93568     
_________________________________________________________________
res_conv1d_34 (ResConv1D)    (None, 430, 128)          116096    
_________________________________________________________________
res_conv1d_35 (ResConv1D)    (None, 430, 256)          330496    
_________________________________________________________________
res_conv1d_36 (ResConv1D)    (None, 430, 512)          1316352   
_________________________________________________________________
res_conv1d_37 (ResConv1D)    (None, 430, 1024)         5254144   
_________________________________________________________________
global_max_pooling1d_4 (Glob (None, 1024)              0         
_________________________________________________________________
dense_14 (Dense)             (None, 256)              

In [15]:
train_model(model, callbacks = callback)

Epoch 1/10
1115/1115 [==============================] - 532s 467ms/step - loss: 3.1508 - accuracy: 0.3392 - val_loss: 2.0455 - val_accuracy: 0.3490
Epoch 2/10
1115/1115 [==============================] - 540s 485ms/step - loss: 1.6904 - accuracy: 0.4370 - val_loss: 2.0237 - val_accuracy: 0.3577
Epoch 00002: early stopping


{'loss': [2.062649965286255, 1.6741598844528198],
 'accuracy': [0.38504764437675476, 0.44227859377861023],
 'val_loss': [2.0455172061920166, 2.023669719696045],
 'val_accuracy': [0.3489583432674408, 0.3576822876930237]}

In [2]:
model = K.models.Sequential([
    K.layers.LSTM(256, return_sequences = True, input_shape = DIM),
    K.layers.LSTM(256),
    K.layers.Dense(256, activation = 'relu'),
    K.layers.Dense(12, activation = 'softmax')
])

In [3]:
train_model(model, callbacks = callback)

Epoch 1/10
1115/1115 [==============================] - 429s 378ms/step - loss: 2.0007 - accuracy: 0.3025 - val_loss: 2.3408 - val_accuracy: 0.1820
Epoch 2/10
 512/1115 [============>.................] - ETA: 2:23 - loss: 2.0310 - accuracy: 0.2977

InternalError:    Failed to call ThenRnnBackward with model config: [rnn_mode, rnn_input_mode, rnn_direction_mode]: 2, 0, 0 , [num_layers, input_size, num_units, dir_count, max_seq_length, batch_size, cell_num_units]: [1, 84, 256, 1, 430, 64, 256] 
	 [[{{node gradients/CudnnRNN_grad/CudnnRNNBackprop}}]]
	 [[Adam/gradients/PartitionedCall_1]] [Op:__inference_train_function_5501]

Function call stack:
train_function -> train_function -> train_function


In [4]:
model = K.models.Sequential([
    K.layers.InputLayer(input_shape = DIM),
    K.layers.Conv1D(128, kernel_size = 3, dilation_rate = 2, activation = 'relu'),
    K.layers.MaxPool1D(),
    K.layers.Conv1D(256, kernel_size = 3, dilation_rate = 2, activation = 'relu'),
    K.layers.MaxPool1D(),
    K.layers.Conv1D(256, kernel_size = 3, dilation_rate = 2, activation = 'relu'),
    K.layers.MaxPool1D(),
    K.layers.GRU(256, return_sequences = True),
    K.layers.GRU(256),
    K.layers.Dense(12, activation = 'softmax')
])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_6 (Conv1D)            (None, 426, 128)          32384     
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 213, 128)          0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 209, 256)          98560     
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 104, 256)          0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 100, 256)          196864    
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, 50, 256)           0         
_________________________________________________________________
gru_4 (GRU)                  (None, 50, 256)          

In [5]:
train_model(model)

Epoch 1/10
1115/1115 [==============================] - 350s 307ms/step - loss: 1.8981 - accuracy: 0.3600 - val_loss: 2.0616 - val_accuracy: 0.3504
Epoch 2/10
1115/1115 [==============================] - 392s 351ms/step - loss: 1.5465 - accuracy: 0.4945 - val_loss: 1.9640 - val_accuracy: 0.3738
Epoch 3/10
1115/1115 [==============================] - 358s 321ms/step - loss: 1.4436 - accuracy: 0.5213 - val_loss: 1.7600 - val_accuracy: 0.4318
Epoch 4/10
1115/1115 [==============================] - 367s 329ms/step - loss: 1.3929 - accuracy: 0.5433 - val_loss: 1.7903 - val_accuracy: 0.4190
Epoch 5/10
1115/1115 [==============================] - 455s 409ms/step - loss: 1.3294 - accuracy: 0.5581 - val_loss: 1.7035 - val_accuracy: 0.4604
Epoch 6/10
1115/1115 [==============================] - 487s 438ms/step - loss: 1.3016 - accuracy: 0.5709 - val_loss: 1.6623 - val_accuracy: 0.4616
Epoch 7/10
1115/1115 [==============================] - 354s 318ms/step - loss: 1.2519 - accuracy: 0.5881 - val_

KeyboardInterrupt: 

In [2]:
class ResConv1D(K.layers.Layer):
    def __init__(self, _output, kernel_size = 3, strides = 1, dilation_rate = 1, padding = 'SAME'):
        super(ResConv1D, self).__init__()
        
        self.conv = K.layers.Conv1D(
            _output,
            kernel_size = kernel_size,
            strides = strides,
            dilation_rate = dilation_rate,
            padding = padding,
            activation = 'relu'
        )
        self.normalization = K.layers.BatchNormalization()
        
        self.T1 = K.layers.Conv1D(
            _output,
            kernel_size = kernel_size,
            padding = padding,
            activation = 'relu'
        )
        self.T_normalization1 = K.layers.BatchNormalization()
        
        self.T2 = K.layers.Conv1D(
            _output,
            kernel_size = kernel_size,
            padding = padding,
            activation = 'relu'
        )
        self.T_normalization2 = K.layers.BatchNormalization()
        
        self.T3 = K.layers.Conv1D(
            _output,
            kernel_size = kernel_size,
            padding = padding,
            activation = 'relu'
        )
        self.T_normalization3 = K.layers.BatchNormalization()
        
        self.upsample = K.layers.Conv1D(
            _output,
            kernel_size = 1,
            strides = strides,
            padding = padding,
            activation = 'relu'
        )
        
    def call(self, parameters):
        result = self.normalization(self.conv(parameters))
        result = self.T_normalization1(self.T1(result))
        result = self.T_normalization2(self.T2(result))
        result = self.T_normalization3(self.T3(result))
        result += self.upsample(parameters)
        return result

In [4]:
model = K.models.Sequential([
    K.layers.Input(shape = DIM),
    ResConv1D(128, strides = 2),
    ResConv1D(256, strides = 2),
    ResConv1D(512, strides = 2),
    ResConv1D(1024, strides = 2),
    ResConv1D(2048, strides = 2),
    K.layers.GlobalMaxPool1D(),
    K.layers.Dense(12, activation = 'softmax')
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
res_conv1d_5 (ResConv1D)     (None, 215, 128)          193152    
_________________________________________________________________
res_conv1d_6 (ResConv1D)     (None, 108, 256)          726272    
_________________________________________________________________
res_conv1d_7 (ResConv1D)     (None, 54, 512)           2894336   
_________________________________________________________________
res_conv1d_8 (ResConv1D)     (None, 27, 1024)          11555840  
_________________________________________________________________
res_conv1d_9 (ResConv1D)     (None, 14, 2048)          46180352  
_________________________________________________________________
global_max_pooling1d (Global (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 12)               

In [ ]:
def change_value(epoch, lr):
    if epoch % 10 == 0:
        lr = lr * 0.1
    return lr

callback = [K.callbacks.LearningRateScheduler(change_value)]

train_model(model, epochs = 100, callbacks = callback)

Epoch 1/100
